In [1]:
input_for_annot_err = "/nfs/research/goldman/zihao/Datas/p1/File_5_annot/Error_pos_for_annot.txt"
input_for_annot_af = "/nfs/research/goldman/zihao/Datas/p1/File_5_annot/AF_2_RATIO.txt"
input_for_annot_sb = "/nfs/research/goldman/zihao/Datas/p1/File_5_annot/SB_2_RATIO.txt"

input_for_cov_err = "/nfs/research/goldman/zihao/Datas/p1/File_5_coverage/Error_pos_for_coverage.txt"
input_for_cov_all = "/nfs/research/goldman/zihao/Datas/p1/File_5_coverage/Cov_RATIO.txt"

## 1. Error position

### 1.1_input SB&AF

In [2]:
import pandas as pd

# 读取数据
df = pd.read_csv(input_for_annot_err, sep='\t')

# 分别计算 DP、AF、SB 的平均值
columns = ["AF", "SB"]
results = {}
for i, col in enumerate(columns):
    df_col = df.dropna(subset=[col])[["Position", col]]
    df_col[col] = df_col[col].astype(float)
    result = df_col.groupby(by=["Position"], as_index=False).max(numeric_only=True)
    result = result[result[col] != 0]  # 只保留非零值
    results[f"df_err_v{i+1}"] = result
    

# 返回结果
df_err_v1 = results["df_err_v1"]
df_err_v2 = results["df_err_v2"]

# 创建一个新的索引
new_index = pd.RangeIndex(start=1, stop=29903, step=1)

# 重新索引数据框
df_err_v1 = df_err_v1.set_index('Position').reindex(new_index, fill_value=0).reset_index(drop=False).rename(columns={'index': 'Position'})
df_err_v2 = df_err_v2.set_index('Position').reindex(new_index, fill_value=0).reset_index(drop=False).rename(columns={'index': 'Position'})

### 1.2_input coverage

In [3]:
df_err = pd.read_csv(input_for_cov_err, sep='\t')
# 使用loc和布尔索引筛选出满足条件的行，然后删除该列
df = df_err[df_err['Position'].astype(int) <= 29903]

df["MEAN_err"] = pd.to_numeric(df_err['MEAN_err'], errors='coerce')
df_err = df.groupby(by=["Position"], as_index=False).mean(numeric_only=True)

# 重新索引数据框
df_err = df_err.set_index('Position').reindex(new_index, fill_value=0).reset_index(drop=False).rename(columns={'index': 'Position'})

df_err

,Position,MEAN_err
0,1,0.000000
1,2,0.002446
2,3,0.004533
3,4,0.000000
4,5,0.000000
...,...,...
29897,29898,0.000000
29898,29899,0.000000
29899,29900,0.000000
29900,29901,0.000000


## Tips 
SB>0, and/or a very low coverage (for example a coverage ratio<0.1), and/or an AF>0.01(need change) .

In [4]:
# Convert the 'MEAN_err' column to a floating point number
df_err['MEAN_err'] = pd.to_numeric(df_err['MEAN_err'], errors='coerce')

# Replace the NaN value with 0
df_err['MEAN_err'] = df_err['MEAN_err'].fillna(0)

# Compare the 'MEAN_err' column with 0.1
df_err['Flag'] = df_err['MEAN_err'] > 0.1

# Compare the 'MEAN_err' column with 0.1
df_err_v1['Flag'] = df_err_v1['AF'] > 0.01

# Compare the 'MEAN_err' column with 0.1
df_err_v2['Flag'] = df_err_v2['SB'] > 0

In [8]:
(df_err[df_err['Flag'] == False]).count()/29902

Position    0.845696
MEAN_err    0.845696
Flag        0.845696
dtype: float64

In [9]:
(df_err_v1[df_err_v1['Flag'] == False]).count()/29902

Position    0.89275
AF          0.89275
Flag        0.89275
dtype: float64

In [10]:
(df_err_v2[df_err_v2['Flag'] == False]).count()/29902

Position    0.895759
SB          0.895759
Flag        0.895759
dtype: float64

In [8]:
df_err.to_csv("/homes/zihao/EBI_INTER/A_Datas/ERR_COV.txt", sep='\t', index=False)
df_err_v1.to_csv("/homes/zihao/EBI_INTER/A_Datas/ERR_AF.txt", sep='\t', index=False)
df_err_v2.to_csv("/homes/zihao/EBI_INTER/A_Datas/ERR_SB.txt", sep='\t', index=False)

## 2. All position

#### 1) For annot

In [12]:
df_all_1 = pd.read_csv(input_for_annot_af, 
                       sep='\t').reset_index().rename(columns={'index': 'POS'})
df_all_2 = pd.read_csv(input_for_annot_sb, 
                       sep='\t').reset_index().rename(columns={'index': 'POS'})
df_all_1['Flag'] = df_all_1['ERROR_all'] > 0.01
df_all_2['Flag'] = df_all_2['ERROR_all'] > 0

In [13]:
(df_all_1[df_all_1['Flag'] == False]).count()/29902

POS          0.976958
ERROR_all    0.976958
Flag         0.976958
dtype: float64

In [14]:
(df_all_2[df_all_2['Flag'] == False]).count()/29902

POS          1.000033
ERROR_all    1.000033
Flag         1.000033
dtype: float64

In [16]:
(df_all_2[df_all_2['Flag'] == False]).count()

POS          29903
ERROR_all    29903
Flag         29903
dtype: int64

#### 2) For coverage

In [ ]:
df_all = pd.read_csv(input_for_cov_all, sep='\t').reset_index().rename(columns={'index': 'POS'})
df_all['Flag'] = df_all['ERROR_all'] > 0.1

In [19]:
(df_all[df_all['Flag'] == False]).count()/29902

POS          0.201692
ERROR_all    0.201692
Flag         0.201692
dtype: float64

In [10]:
df_all_1.to_csv("/homes/zihao/EBI_INTER/A_Datas/ALL_AF.txt", sep='\t', index=False)
df_all_2.to_csv("/homes/zihao/EBI_INTER/A_Datas/ALL_SB.txt", sep='\t', index=False)
df_all.to_csv("/homes/zihao/EBI_INTER/A_Datas/ALL_COV.txt", sep='\t', index=False)

### Next 
- go ---> May.2_Venn.Rmd